# 使用するカタログとスキーマを指定

In [0]:
USE CATALOG handson;
USE SCHEMA sports_orders;

# ゴールドテーブル作成（初回1回のみ）

In [0]:
CREATE TABLE IF NOT EXISTS gold_sports_summary (
  customer_name STRING,
  total_quantity INT,
  total_sales DOUBLE,
  first_order_date DATE,
  last_order_date DATE
);

# シルバーデータから集計後にMERGE

In [0]:
MERGE INTO gold_sports_summary AS target
USING (
  SELECT
    customer_name,
    SUM(quantity) AS total_quantity,
    SUM(quantity * price) AS total_sales,
    MIN(order_date) AS first_order_date,
    MAX(order_date) AS last_order_date
  FROM silver_sports_orders
  GROUP BY customer_name
) AS source
ON target.customer_name <=> source.customer_name -- NULLの重複を回避（NULLセーフな比較演算子）
WHEN MATCHED THEN
  UPDATE SET
    target.total_quantity = source.total_quantity,
    target.total_sales = source.total_sales,
    target.first_order_date = source.first_order_date,
    target.last_order_date = source.last_order_date
WHEN NOT MATCHED THEN
  INSERT *
;